### Installer les bibliothèques

In [ ]:
%pip install semantic-link
%pip install pyspark
%pip install azure-keyvault-secrets==4.7.0
%pip install azure-identity
%pip install azure-mgmt-resource
%pip install azure-storage-blob
%pip install azure-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.5/284.5 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=4647fd01d656262fd891ba71df164f03ea8aadd3e95714918623bdbfb024d53d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.6/348.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 7.3 MB/s eta 0:00:00
  Attempting unin

In [ ]:
import pandas as pd
import sempy.fabric as fabric
from sempy.fabric import FabricDataFrame
from sempy.dependencies import plot_dependency_metadata
import re

### Préparation des données

In [ ]:
data_emp = """
PROJ_NUM PROJ_NAME EMP_NUM EMP_NAME CHG_HOUR JOB_CLASS HOURS
15 Evergreen 103 June E. Arbough 84.50 Elect. Engineer 23.8
15 Evergreen 101 John G. News 105.00 Database Designer 19.4
15 Evergreen 105 Alice K. Johnson 105.00 Database Designer 35.7
15 Evergreen 106 William Smithfield 35.75 Programmer 12.6
15 Evergreen 102 David H. Senior 96.75 Systems Analyst 23.8
18 Amber Wave 114 Annelise Jones 48.10 Applications Designer 24.6
18 Amber Wave 118 James J. Frommer 18.36 General Support 45.3
18 Amber Wave 104 Anne K. Ramoras 96.75 Systems Analyst 32.4
18 Amber Wave 112 Darlene M. Smithson 45.95 DSS Analyst 44.0
22 Rolling Tide 105 Alice K. Johnson 105.00 Database Designer 64.7
22 Rolling Tide 104 Anne K. Ramoras 96.75 Systems Analyst 48.4
22 Rolling Tide 113 Delbert K. Joenbrood 48.10 Applications Designer 23.6
22 Rolling Tide 111 Geoff B. Wabash 26.87 Clerical Support 22.0
22 Rolling Tide 106 William Smithfield 35.75 Programmer 12.8
25 Starflight 107 Maria D. Alonzo 35.75 Programmer 24.6
25 Starflight 115 Travis B. Bawangi 96.75 Systems Analyst 45.8
25 Starflight 101 John G. News 105.00 Database Designer 56.3
25 Starflight 114 Annelise Jones 48.10 Applications Designer 33.1
25 Starflight 108 Ralph B. Washington 96.75 Systems Analyst 23.6
25 Starflight 118 James J. Frommer 18.36 General Support 30.5
25 Starflight 112 Darlene M. Smithson 45.95 DSS Analyst 41.4
25 Starflight 95 Darlene M. Smithson 96.75 Systems Analyst 23.6
"""

all_rows_new = []

for line in data_emp.split('\n')[1:]:
  # Use regular expression to match the pattern of each line
  match = re.match(
      r'(?P<PROJ_NUM>\d+)\s+'  # Project number: one or more digits
      r'(?P<PROJ_NAME>[\w\s]+?)\s+'  # Project name: word characters including spaces, non-greedy
      r'(?P<EMP_NUM>\d+)\s+'  # Employee number: one or more digits
      r'(?P<EMP_NAME>[\w\s.]+?)\s+'  # Employee name: word characters including spaces and dots, non-greedy
      r'(?P<CHG_HOUR>\d+\.\d+)\s+'  # Charge per hour: one or more digits, a dot, one or more digits
      r'(?P<JOB_CLASS>[\w\s.]+?)\s+'  # Job class: word characters including spaces and dots, non-greedy
      r'(?P<HOURS>\d+\.\d+)',  # Hours: one or more digits, a dot, one or more digits
      line.strip()
  )

  if match:
      row_data = match.groupdict()
      all_rows_new.append(row_data)

df = pd.DataFrame(all_rows_new)
df["CHG_HOUR"] = df["CHG_HOUR"].astype(float)
df["HOURS"] = df["HOURS"].astype(float)
df["EMP_NUM"] = df["EMP_NUM"].astype(int)
df["PROJ_NUM"] = df["PROJ_NUM"].astype(int)
df.head(22)

,PROJ_NUM,PROJ_NAME,EMP_NUM,EMP_NAME,CHG_HOUR,JOB_CLASS,HOURS
0,15,Evergreen,103,June E. Arbough,84.50,Elect. Engineer,23.8
1,15,Evergreen,101,John G. News,105.00,Database Designer,19.4
2,15,Evergreen,105,Alice K. Johnson,105.00,Database Designer,35.7
3,15,Evergreen,106,William Smithfield,35.75,Programmer,12.6
4,15,Evergreen,102,David H. Senior,96.75,Systems Analyst,23.8
5,18,Amber Wave,114,Annelise Jones,48.10,Applications Designer,24.6
6,18,Amber Wave,118,James J. Frommer,18.36,General Support,45.3
7,18,Amber Wave,104,Anne K. Ramoras,96.75,Systems Analyst,32.4
8,18,Amber Wave,112,Darlene M. Smithson,45.95,DSS Analyst,44.0
9,22,Rolling Tide,105,Alice K. Johnson,105.00,Database Designer,64.7


### 1) Utilisez ChatGPT pour créer une fonction qui donne les clés candidates de cette relation

In [1]:
### SOLUTION ICI

### 2) Utilisez ChatGPT pour créer deux fonctions : une qui renvoie les attributs premiers, et l'autre les attributs non premiers

In [ ]:
### SOLUTION ICI

### 3) Utilisez ChatGPT pour créer une fonction qui vérifie s'il y a des dépendances partielles et les affiche

In [ ]:
### SOLUTION ICI

### 4) Utilisez ChatGPT pour créer une fonction qui vérifie s'il y a des dépendances transitives et les affiche

In [ ]:
### SOLUTION ICI

### 5) a) Lister toutes les dépendances fonctionnelles dans cette relation

### b) Expliquez pourquoi cette relation est sous la première forme normale.

### 6) a) Expliquez pourquoi cette relation n'est pas sous la deuxième forme normale.
### b) Convertissez cette relation en 2FN.

### 7) a) Expliquez pourquoi cette relation n'est pas dans la troisième forme normale.
### b) Convertissez cette relation en 3FN.

### 8) Fournissez le script SQL pour créer les tables sous la troisième forme normale en plus du script d'insertion.

#### Vous pouvez utiliser ChatGPT pour générer les scripts ci-dessus.